<a href="https://colab.research.google.com/github/kodeu/trading-lstm/blob/model_setup/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf

num_data_points = 100  # Total data points
feature_size = 5       # Number of features per timestep
window_size = 20       # Maximum sequence length

# Generate dummy feature data and targets
data = np.random.rand(num_data_points, feature_size)
targets = np.random.rand(num_data_points, 1)

# Create sequences with a maximum length of 'window_size'
X = [data[max(0, i-window_size+1):i+1] for i in range(num_data_points)]
y = [targets[max(0, i-window_size+1):i+1] for i in range(num_data_points)]

print(X[:3])

# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_generator(
    lambda: ((X[i], y[i]) for i in range(len(X))),
    output_types=(tf.float32, tf.float32),
    output_shapes=((None, feature_size), (None, 1))
)

# Batch, pad and prefetch
train_dataset = train_dataset.padded_batch(1, padded_shapes=([None, feature_size], [None, 1])).prefetch(tf.data.experimental.AUTOTUNE)


[array([[0.13527738, 0.85625771, 0.98748557, 0.63543015, 0.64728419]]), array([[0.13527738, 0.85625771, 0.98748557, 0.63543015, 0.64728419],
       [0.97110574, 0.93583211, 0.34795809, 0.66531915, 0.52889945]]), array([[0.13527738, 0.85625771, 0.98748557, 0.63543015, 0.64728419],
       [0.97110574, 0.93583211, 0.34795809, 0.66531915, 0.52889945],
       [0.54497896, 0.0656654 , 0.59563736, 0.0942389 , 0.15494837]])]


In [3]:
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Attention
from tensorflow.keras.models import Model

# Model setup
input_layer = Input(shape=(None, feature_size))
lstm_layer = LSTM(50, return_sequences=True)(input_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])
output_layer = TimeDistributed(Dense(1))(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 5)]            0         []                            
                                                                                                  
 lstm (LSTM)                 (None, None, 50)             11200     ['input_1[0][0]']             
                                                                                                  
 attention (Attention)       (None, None, 50)             0         ['lstm[0][0]',                
                                                                     'lstm[0][0]']                
                                                                                                  
 time_distributed (TimeDist  (None, None, 1)              51        ['attention[0][0]']       

In [4]:
# Train the model
history = model.fit(train_dataset, epochs=10, verbose=1)


Epoch 1/10
100/100 [==============================] - 3s 10ms/step - loss: 0.0902
Epoch 2/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0862
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0852
Epoch 4/10
100/100 [==============================] - 1s 8ms/step - loss: 0.0844
Epoch 5/10
100/100 [==============================] - 1s 8ms/step - loss: 0.0835
Epoch 6/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0829
Epoch 7/10
100/100 [==============================] - 1s 10ms/step - loss: 0.0820
Epoch 8/10
100/100 [==============================] - 1s 13ms/step - loss: 0.0822
Epoch 9/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0818
Epoch 10/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0814
